<a href="https://colab.research.google.com/github/annikaaross/Homochirality-project/blob/precursor%2Fautocatalyze/Homochirality_with_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homochirality Model

This model is designed to keep track of a pool of monomers and polymers. It starts with a pool of monomers. In each itertion it bonds them or breaks them. Once two monomers are bonded they become a polymer. The probability that a specific bond of a polymer will break is determined by whether or not the bond is homochiral and the amount of homochiral bonds that surround the bond. The model tests the impact of the strength of homochirality in bonds on the overall chirality of the pool of reactables. 

## Imports and Helpers


First we install a package called ```jdc``` which allows us to define classes across different cells using the syntax

```
%%add_to our_class
def our_function(self, our_variable):
  print our_variable
```

The documentation on this magic function can be found at https://alexhagen.github.io/jdc/






In [21]:
#@title jdc install [Click play to run. Double click here to see/edit the code]
!pip install jdc

Here are the imports.

In [22]:
#@title Imports [Click play to run. Double click here to see/edit the code]
import random
import numpy as np
import copy
from google.colab import widgets, output
import matplotlib.pyplot as plt
import jdc
import more_itertools
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
import datetime
import uuid
from IPython.display import display, Markdown, clear_output
import ipywidgets
import glob

In [23]:
#@title Note: We need to mount Drive to be able to export and store data. To skip, uncheck the box below. 
# Checkbox to en-/dis-able exporting to Google Drive
w_export_to_drive = ipywidgets.Checkbox(
    value=True,
    description='Export to Drive',
)
display(w_export_to_drive)

Checkbox(value=True, description='Export to Drive')

In [24]:
#@title Mount Drive [click play to run, double-click to show/hide code]

if w_export_to_drive.value == True:
  from google.colab import drive
  drive.mount('/content/drive')
  wdir = '/content/drive/Shared drives/Homochirality/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


And here are some helper functions.

In [25]:
#@title Helper Functions and Vars [double-click here to see/edit code]
def make_pool(n):
  """
  Return a list of n new monomers.
  """
  monomers = []
  for i in range(n):
    monomers.append(Monomer())
  return monomers


# def numpy_fillna(data):
#   """ Rectangularize a jagged array.

#   Source: https://stackoverflow.com/a/32043366
#   """
#   # Get lengths of each row of data
#   lens = np.array([len(i) for i in data])

#   # Mask of valid places in each row
#   mask = np.arange(lens.max()) < lens[:,None]

#   # Setup output array and put elements from data into masked positions
#   out = np.zeros(mask.shape, dtype=float)
#   out[mask] = np.concatenate(data)
#   return out

def eAnd(*args): # From https://stackoverflow.com/q/2770434
  return [all(tuple) for tuple in zip(*args)]

def get_rand():

  """A function to provide uniform random numbers"""
  
  if len(rands)<= 10:
    #refills the array if contains less than 10
    randsarray = np.random.rand(N_RANDS)
    rands.extend(randsarray.tolist())
  #return last number and deletes it from the array
  return rands.pop()

def poissonequation(k):

  """Given a value k, returns the result of the poisson equationn for k"""
  
  #checks if value has already been calculated
  if k not in poisson_dict:
    #otherwise calculates the values
    p = np.exp(-LAMBDA)*((LAMBDA**k)/(np.math.factorial(k)))
    #adds to dictionary
    poisson_dict[k] = 1- 1.3*(p)
  return poisson_dict[k]
  
def dict_to_csv(params,filename):
  with open(filename, 'w') as f:
    for key in params.keys():
      f.write("%s,%s\n"%(key,params[key]))
  f.close()


## The Monomer class

The monomer class holds monomer objects. Monomer objects have a handedness that is stored as a boolean value, a bond break probability, and an age, how many iterations they have existed in the simulation.

In [26]:
#@title Constructor

class Monomer:

  def __init__(self, **kwargs):
    """
    The constructor method for Monomer objects. Assigns default handedness.
    Accepts argument hand = bool with keyword.
    Unless provided, assigns easbrkprob to -1 (indicating it is not in a polymer).
    Initializes age, how many iterations the monomer has survived, as 0.
    """

    self._handedness = kwargs['hand'] if 'hand' in kwargs else random.choice([True, False])
    self._eastbrkprob = kwargs['brkprob'] if 'brkprob' in kwargs else -1
    self._age = 0
    self._uuid = uuid.uuid4()
  
  def __repr__(self):
    """repr function for Monomer Class"""
    return str(f"{self.get_handedness()}-monomer")


In [27]:
#@title Getters and Setters
%%add_to Monomer

def get_handedness(self):
  """Getter method for Monomer handedness."""
  return self._handedness

def get_age(self):
  """Getter method for Monomer age"""
  return self._age


def get_eastbrkprob(self):
  """Getter method for east bond brk probability"""
  return self._eastbrkprob

def set_eastbrkprob(self, newbrk):
  """Setter method for east bond brk probability"""
  self._eastbrkprob = newbrk

def set_handedness(self,newhandedness):
  """setter method for Monomer handedness."""
  self._handedness = newhandedness


def set_age(self, newage):
  """setter method for age"""
  self._age = newage

def reset_eastbrkprob(self) :
  """reset east brk probability to -1 
      to be called when single monomer is broken from polymer"""
  self.set_eastbrkprob(-1)

def get_id(self):
  return self._uuid


In [28]:
#@title Polymer Compatibility
%%add_to Monomer

def get_sequence(self):
  return (self.get_handedness(),)

def birthday(self):
  """ages the monomer up"""
  self._age += 1

def reset_eastbrkprob(self) :
  """
  reset east brk probability to -1 
  to be called when single monomer is broken from polymer
  """
  self.set_eastbrkprob(-1)

## The Polymer class

The polymer class stores polymer objects. Polymer objects are lists of the monomer objects which they contain. The lists are kept in a specific order. The polymer objects also have ages, the number of iteraions they have existed in the simulation. The class has methods that allow the addition of new monomers to a polymer(this can either be done by adding a single monomer or all the monomers from another polymer), remove all monomers after a specific index, detect homochirality in a bond and use this to calculate probability of a certain bond breaking, and there is a method which uses these break probabilities to randomly decide if and where a polymer is going to break.

In [29]:
#@title Constructor

class Polymer:
  def __init__(self, monomers = []):
    """ 
    Constructor method for Polymer. 
    Sets list of monomers it contains.
    Keeps track of the age of itself (number of iterations it has survived)
    """
    self._monomers = monomers
    self._age = 0
    self._uuid = uuid.uuid4()
 
  def __str__(self):
    """ str function for Polymer class. """
    return f"\nA polymer of length {self.get_length()}: {self.get_sequence()}"
  
  def __repr__(self):
    """ repr function for Polymer class. """
    return f"{self.get_sequence()}-polymer"

In [30]:
#@title Getters, Setter, and Logs
%%add_to Polymer


def get_monomers(self):
  """ Return list of monomers in the polymer. """
  return self._monomers


def get_length(self):
  """ Return the number of monomers in the polymer. """
  return len(self._monomers)


def get_sequence(self):
  """ Return the sequence of monomers in the polymer in a human-readable (and loggable) format. """
  sequence = []
  for monomer in self.get_monomers():
    sequence.append(monomer.get_handedness())
  return sequence

def get_age(self):
  """
  Getter method for the age of the polymer
  """
  return self._age


def set_age(self, newage):
  """
  Setter method for the age of the polymer
  """
  self._age = newage

def set_monomers(self, new_monomers):
  """ Set the monomers in the polymer to new_monomers. """
  self._monomers = new_monomers

def birthday(self):
  """
  Ages up the polymer and all of the monomers in the polymer
  """
  #ages up itself
  self._age += 1
  #ages up all its monomers
  for n in range(self.get_length()):
    self.get_monomers()[n].birthday()

def get_id(self):
  return self._uuid


In [31]:
#@title Functionality Methods
%%add_to Polymer

def append(self, other):
  """
  Adds a reactable (monomer or polymer) to the end of the Polymer.
  If the added reactable is a polymer, this method deletes it after taking its monomers.
  """
  #Check the type of the thing to be added

  if isinstance(other, Monomer):
    #Monomers can just be appended to the list
    self._monomers.append(other)

  elif isinstance(other, Polymer):
    #For polymers we need to extend rather than append, to avoid nested lists
    self._monomers.extend(other.get_monomers())
    #Then once the monomers are safely in their new polymer, the old one is deleted
    del other

def brkloc(self):

  """This method randomly goes through the bonds of a polymer
  and asks each if it will brk, returns first that will, or None if none break"""

  #list of all the indices of monomers in the polymer except for the last one
  indices = []
  for n in range(self.get_length() - 1):
    indices.append(n)

  #puts indices in random order
  random.shuffle(indices)

  #tests if each index will break
  for index in indices:
    #gets the breakprobability for monomer at n index
    brkprob = self._monomers[index].get_eastbrkprob()
    #pulls a random number
    rand = get_rand()
    # checks if the random< brkprob of index
    if(brkprob > rand):
      #if so returns index
      return index
  #if none break, return none
  return None

def removeeast(self, location):
  """
  Removes all monomers to the right (east) of a given index
  and returns a list of the monomers that were removed
  """
  #list of the monomers at the, and to the left of
  #the location provided
  newList = self._monomers[0:location+1]

  #list of the monomers to the right of
  #the location provided
  removed = self._monomers[location+1:]

  #resets monomers to only include those to the left
  self.set_monomers(newList)

  #returns the monomers that were removed
  return removed

In [32]:
##@title Homochirality handling
%%add_to Polymer

def easthomochiralcheck(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer and returns whether its east bond is homochiral.
  """
  #First check that the index isn't out of bounds 
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1):
    return False
  #get the handedness of this monomer and its east neighbor, and return whether or not they're equal (bool)
  return self._monomers[numbermonomer].get_handedness() == self._monomers[numbermonomer+1].get_handedness()

def easthomochiralbiascheck (self,numbermonomer):
  """ this method can be used in place of the east homochiral check method
  returns false if not homochiral, 3 if homochiral left, 5 if homochiral right"""

  #First check that the index isn't out of bounds or if not homochiral
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1 or not self.easthomochiralcheck(numbermonomer)):
    return False
  
  #otherwise check if they are homochiral left or right
  elif self._monomers[numbermonomer].get_handedness():
      #means left homochiral
      return 3
  elif not self._monomers[numbermonomer].get_handedness():
      #means right homochiral
      return 5
  

def eastbondbreakprobability(self,numbermonomer,basebrk):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
    
  #now we initialize brk probability (brk)
  brk = basebrk
  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so multiply it by homochiral break factor (shrinks probability)
    brk *= HOMOCHIRAL_BREAK_FACTOR
    #goes through method which checks and calculates benefit of all homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR)
  #end def: returns break probability of monomers east bond
  return brk

def biaseastbondbreakprobability(self,numbermonomer,basebrk):
  """
  ***left-right sensitivity****
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  #initialize the brk probability
  brk = basebrk
  #check if the east bond is homochiral left
  if (self.easthomochiralbiascheck(numbermonomer) == 3):
    brk *= HOMOCHIRAL_BREAK_FACTOR_LEFT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer, brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT)

  #otherwise checks if bond is homochiral right
  elif (self.easthomochiralbiascheck(numbermonomer) == 5):
    brk *= HOMOCHIRAL_BREAK_FACTOR_RIGHT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT)
   
  return brk

def checkforhomochiralneighbors(self, numbermonomer, brk, neighborfactor):

  """
  helps all the versions of the eastbond break calculating functions
  takes in a brk probability a number monomer and the desired neighbor improvement factor
  and calculates how the neighbors a bond will increase the strength of the bond
  it then returns the new brk prob to the eastbond brk function
  """
  brk = brk
  j = numbermonomer + 1
  #going right to check for homochiral neighbors
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1 - (neighborfactor**abs(j - numbermonomer))
    j += 1

  #going left to check for homochiral neighbors
  j=numbermonomer-1
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1-(neighborfactor**abs(numbermonomer - j))
    j -= 1

  return brk


def poissonbreakprobability(self, numbermonomer, basebrk):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  based on Poisson distribution
  returns -3 if the monomer has no east bond
  """
  #With that out of the way, we initialize brk to the base probability of breaking a bond
  brk = basebrk
  #keeps track of the number of the total consecutive homochiral bonds
  homochiralcount=0 

  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so homochiral count increases by 1
    homochiralcount +=1
    #brk gets benefited by the poisson distribution of homochiral count 
    brk *= poissonequation(homochiralcount)

    #set j to monomer to right
    j = numbermonomer+1

    #going right to check for neighboring homochirality
    while(self.easthomochiralcheck(j)):
      #if homochiral increases count
      homochiralcount += 1
      #recalculates brk prob
      brk *= poissonequation(homochiralcount)

      #check next monomer
      j += 1

    #going left to check for neighboring homochirality
    j=numbermonomer-1
    #homochiralcount = 1
    while(self.easthomochiralcheck(j)):
      homochiralcount += 1
      brk *= poissonequation(homochiralcount)
      j -= 1
  return brk


def reset_break_probability(self, method):
    
  """ 
  resets the break probabilities of the monomers in a 
  sequence. 1)checks lookup table to see if similar polymer values
  have been caluclated, otherwise calculates values and adds
  polymer to lookup table 2)sets values to monomers.
 
  """
  #store self as list of booleans to be added to lookup table
  polyseq = self.get_monomers().copy()
  for n in range(self.get_length()):
    polyseq[n] = polyseq[n].get_handedness()

  ##########################
  # LEFT/RIGHT SENSITIVITY #
  ##########################
  # All stored sequences start with True. If the one being requested doesn't, invert so that it does
  if not method == "bias":
    if polyseq[0] == True:
      polyseq = [not mono for mono in polyseq]

  #make polyseq a tuple able to be added as an index of a dictionary  
  polyseq = tuple(polyseq)

  # If the sequence is already in the table (dict), retrieves its previously calculated break probabilities
  if polyseq in break_prob_lookup_table:
    the_probs = break_prob_lookup_table[polyseq]
    
  else: # Otherwise calculate and add to the lookup table
    # Calculate the break probabilities
    the_probs = self.calculatebrkprob(method)
    #and add it to the lookup table
    break_prob_lookup_table[polyseq] = the_probs

  #set monomers eastbond brk probs to correlated values stored in the_probs
  for n in range (self.get_length()):
    self._monomers[n].set_eastbrkprob(the_probs[n])

  #end def: polymer break probabilities recalculated and added
  #to lookup table (if not already there)

def calculatebrkprob(self, method):
    
  """
  generates and returns a list of break probabilities for the monomers at each index of
  a polymer. This method can be altered to calculate normally
  with left/right sensitivity or with a poisson distribution
  
  Calculation methods: [bias, standard, poisson]

  """

  #initialize brk_probs as an empty list
  brk_probs = []

  #checks if polymer is of length one -meaning it is lone monomer
  if self.get_length() == 1 :
    #if so adds -1 to list(brk prob value assigned to individual monomers)
    brk_probs.append(-1)
    
  else:

    #now we initialize brk probability (brk) for all bonds dependent of chirality
    brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

    #otherwise goes through all the indices of the polymer (except for last monomer),calculates its brk probability, and appends it to brk_probs 
    for n in range(self.get_length()-1):

      #############################################################
      #LEFT RIGHT SENSITIVITY/POISSON EQUATIONS CHOICE.           #
      #############################################################
      if method == "bias":
        brk_probs.append(self.biaseastbondbreakprobability(n, brk))

      elif method == "standard":
        brk_probs.append(self.eastbondbreakprobability(n, brk))

      elif method == "poisson":
        brk_probs.append(self.poissonbreakprobability(n, brk))

      else:
        raise ValueError(f"'{method}' is not a recognized calculation method. Please use one of ['bias', 'standard', 'poisson'].")
    #adds -3 to end of list to represent last monomers east bond break probability(ther is no bond so it is negative)
    brk_probs.append(-3)
  #returns in order list of brk probs for the east bond of all the monomers in a polymer
  return brk_probs


## The Reactables class

This is the Reactables class. The Reactables class stores a list of all the reactables in the simulation (polymers and monomers). If a monomer is in a polymer, it is not individually included in the list of reactables. The reactables class is responsible for carrying out an iteration. In doing so it uses monomer and polymer methods to update them (break them, bond them, age them up, etc.), and edits the reactables list. Additonally in each iteration, it records the state of the current reactable bag in the models history(stored as a pd.dataframe).

In [33]:
#@title Constructor
class Reactables:

  def __init__(self, reactables = []):
    """
    Constructor class for Reactables object
    Accepts list of reactables or defaults to empty list
    Reactables functions as a bin for reactable objects monomer and polymer
    Methods consist mostly of list handling
    """
    self._reactables = reactables
    #The reactables bag also handles remembering its history
    self._history = []
    self._lookup = {}
    self._hist_stats = pd.DataFrame()


  def __str__(self):
    """
    str method for Reactables class
    """
    return str(self.get_reactables())

In [34]:
#@title Getters and Setters
%%add_to Reactables

#################
#GETTERS/SETTERS#
#################

def get_reactables(self):
  """
  Getter method for reactables in Reactables
  Returns a list of the objects currently in the Reactables bag
  """
  return self._reactables

def get_history(self):
  """ Return the history of the reactables bag.
  """
  return self._history

def set_history(self, new_hist):
  """ Set the history to now_hist. """
  self._history = new_hist
  
def get_count(self):
  """
  Getter method for Reactables class
  Returns the number of objects in the Reactables bag
  """
  return len(self.get_reactables()) 


def set_reactables(self, new_list):
  """
  Setter method for the Reactables class
  Sets the reactables bag to a new list
  Used mostly as a helper function in bonding
  """
  self._reactables = new_list


def get_stats(self):
  return self._hist_stats

def set_hist_stats(self,df):
  self._hist_stats = df

def get_lookup(self):
  return self._lookup

def add_stat_lookup(self, key, value):
  self._lookup[key] = value

def add_log(self, log):
  (self._history).append(log.copy())


### Functionality methods

In [35]:
#@title Top level functionality
%%add_to Reactables

def refill(self, pool_size):

  """Method that refills the pool size after each iteration"""

  #initialize a number to record number of monomers to be added to the bag
  number = 0
  #calculate the difference between the pool size and the actual size
  dearth = pool_size - self.get_count()
  #if reactant size is less than pool size
  if dearth  > 0:

    #if refill random method is selected
    if REFILL[0] == "refill random":
      #number = random percentage of the dearth
      number = int(get_rand() * dearth) 

    #if refill percent is selected
    elif REFILL[0] == "refill percent":
      #refill by selected percent of dearth 
      number = int(REFILL[1] * dearth)

    #if refill number is selected
    elif REFILL[0] == "refill number":
      #refill selected number of monomers
      number = REFILL[1]

    #if refill number decrease
    elif REFILL[0] == "refill number decrease":
      #refill recorded number
      number = REFILL[1]
      #if number is >0
      if REFILL[1] > 0:
        #decrease by 1
        REFILL[1] -= 1
    #if refill normal
    elif REFILL[0] == "refill normal":
      #refill dearth
      number = dearth
    #if refill percent decrease
    elif REFILL[0] == "refill percent decrease":
      #refill recorded percent of difference
      number = int(REFILL[1] * dearth)
      #if percent is greater than 0
      if REFILL[1]> 0:
        #decrease by .01
        REFILL[1] -= .01
    
    #loop through number of monomers to be added
    for n in range(number):
      #add a monomer to the reacables bag
      self.add([Monomer()]) 

def randomize_reactables(self):
  """
  Randomizes the order of the reactables list
  """
  #Get the current reactables
  reactables = self.get_reactables()
  #Shuffle them
  random.shuffle(reactables)
  #And reset the reactables list to the new sorted list
  self.set_reactables(reactables)

def do_the_thing(self, method):
  """
  Handle a single iteration of the reactables.

  """
  #We keep track of bonding by storing the most recent reactable to choose to bond as the 'bachelor.'
  bachelor = None
  #We need a copy of the reactables because indices will change as soon as we start doing stuff
  reactables = copy.copy(self.get_reactables())
  #Iterate through each reactable
  for item in reactables:
    #Roll a random number in (0,1) to compare against our probabilities
    roll = get_rand()
    #If our reactable is a monomer...
    if isinstance(item, Monomer):
      #We get our breaking and bonding chances (parameters now, may become functions later?)
      brk = POOF_CHANCE
      bond = BOND_PROB
      #If we roll to break the monomer
      if 0 < roll <= brk:
        #Just delete it
        item.set_handedness(None)
      #If we roll to bond the monomer
      elif brk < roll <= brk+bond:
        #Check if there's a reactable waiting to bond
        if bachelor == None:
          #If not, make this monomer the bachelor. Someone will come along to bond later.
          bachelor = item
        else:
          #If there is someone waiting, bond them together and reset the bachelor chair to empty.
          self.bond_pair(bachelor, item, method)
          bachelor= None
      else:
        continue
    #If the reactable is a polymer instead...
    elif isinstance(item, Polymer):
      #Choose whether the polymer will break or bond. It's 50-50 right now.
      if roll >= 0.33:
        #This is the same bonding logic as for the monomer. It's not in a helper function because it needs to be able to access bachelor.
        if bachelor == None:
          bachelor = item
        elif (type(bachelor) != Polymer or type(item) != Polymer):
          #print("I have gone to bonnnnnnddddddd!!!!")
          #print(bachelor)
          #print(item)
          self.bond_pair(bachelor, item, method)
          bachelor = None
        elif FUSION:
          self.bond_pair(bachelor,item, method)
          bachelor = None
      #Here's what happens if the polymer is chosen to check breaking
      else:
        #It finds its break location
        break_spot = item.brkloc()
        #Which might turn out to be nowhere.
        if break_spot != None:
          #If somewhere does break, call break_polymer to handle the breaking
          self.break_polymer(item,break_spot,method)
    else:
      #You never know what might end up in your reactables bag
      raise ValueError("This thing is neither a monomer or a polymer. What?!")

def iterate(self,size,iteration_number,method):
  """
  Handles full sequence of iteration
  """
  self.randomize_reactables()
  self.do_the_thing(method)
  self.log(iteration_number)
  self.ageup()
  self.refill(size)

def simulate(self, poolsize, iterations, method="standard"):
  for n in range(iterations):
    self.iterate(poolsize, n, method)
  self.parse_history()


In [36]:
#@title Functionality helpers
%%add_to Reactables


def add(self, new_reactables):
  """
  Adds a list of reactables to the reactables bag
  Built on extend(); argument must be iterable
  """
  
  self.get_reactables().extend(new_reactables)
  #print("reactables list is now",self.get_reactables())


def subtract(self, removables):
  """
  Removes a list of reactables from the reactables bag
  Built on list comprehension; argument must be iterable
  """
  self.set_reactables([reactable for reactable in self.get_reactables() if not (reactable in removables)])


def bond_pair(self, west, east, method):
  """
  Accepts two reactables from the bag
  Bonds them together, handling deletion of any emptied polymer
  """

  #this is my attempt to get the ages working properly the idea is:

  #if both are polymers or both are monomers:
  if (type(west) == type(east) and type(west) == Polymer):
    #set the new age to the maximum age of the polymers
    age = max(west.get_age(), east.get_age())
  
  if (type(west) == type(east) and type(west) == Monomer):
    #set age to 0 it is a brand new polymer
    age = 0

  #if only one is a polymer:
  elif isinstance(west, Polymer) and isinstance(east, Monomer):
    #set the age to the age of the polymer
    if east.get_handedness() == None:
      #assigning code::
      age = west.get_age()
  elif isinstance(west, Monomer) and isinstance(east, Polymer):
    # set the age to the age of the polymer
    age = east.get_age()

  #Take your two reactables and check if the west one is a monomer or a polymer 
  if isinstance(west, Polymer):
    #If it's a polymer, all you've got to do is append the east item and then take it out of the bag
    west.append(east)
    self.subtract([east])
    #recalculate the breakprobabilities of the bonds in polymer
    west.reset_break_probability(method)
    #set new polymer to age that was calculated before
    west.set_age(age)

  if isinstance(west, Monomer):
    #If west friend is a Monomer, then make a new polymer containing west
    newpoly = Polymer([west])
    #Add the east things to it
    newpoly.append(east)
    #Remove both the west monomer and the east thing
    self.subtract([east])
    self.subtract([west])
    #And put the new polymer into the reactables bag
    self.add([newpoly])
    #recalculates break probability
    newpoly.reset_break_probability(method)
    #set new polymer to age that was calculated before
    newpoly.set_age(age)


def break_polymer(self, polymer, brk_location, method):
  """
  breaks polymer at given location and creates a new polymer of the 
  monomers removed
  """
  age = polymer.get_age()
  #when polymer is made-breakprobabilities are calculated
  newPolymer = Polymer(polymer.removeeast(brk_location))
  polys = [newPolymer, polymer]
  #goes throught the two new polymers
  for poly in polys:
    #resets break probabilities in polymers
    poly.reset_break_probability(method)
    #sets age to that of the oldest polymer
    poly.set_age(age)

    #checks if they are of length 1(monomer)
    if poly.get_length() <= 1:
      #adds them to reactable as a single monomer
      poly.get_monomers()[0].set_handedness(None)
      print(polymer.get_monomers()[0].get_handedness)
      self.add(poly.get_monomers())
      #subtracts polymer from reactable list
      self.subtract([poly])
      del poly
    #checks if the polymer is not in the reactables list
    elif (poly not in self.get_reactables()):
      #add polymer to reactable list
      self.add([poly])

def ageup(self):
  """
  Method that ages up every reactable in the reactables bag
  """
  for reactable in self.get_reactables():
    reactable.birthday()

    



### History handling methods

In [37]:
#@title Log method
%%add_to Reactables

def log(self, iteration):
  """ 
  Translate current reactables into a loggable list of tuples.
  This log keeps the full sequences of the polymers without digesting the data, 
  so it should be more flexible as we more forward with pulling new information
  out of this simulation. 
  There is one major piece of information lost, however, which is the identities
  of the individual polymers and monomers. 
  """

  # Column labels for History Handling #
  Type = "Type"                   # Monomer or polymer
  Length = "Length"               # How many monomers in the thing
  nLefts = "#Lefts"               # How many left monomers in the thing
  nRights = "#Rights"             # How many right monomers in the thing
  Sequence = "Sequence"           # The string sequence of the thing
  nLhomo = "#LeftHomochiral"      # The number of left homochiral bonds in the polymer
  nRhomo = "#RightHomochiral"     # The number of right homochiral bonds in the polymer
  nHomo = "#Homochiral"           # The overall number of homochiral bonds in the polymer
  sEE = "Signed ee"               # The signed enantiomeric excess of the polymer (+ if more True)
  pcHomo = "%Homochirality"       # The proportion of bonds in the polymer that are homochiral
  pcLhomo = "%LeftHomochirality"  # The proportion of bonds in the polymer that are left homochiral
  pcRhomo = "%RightHomochirality" # The proportion of bonds in the polymer that are right homochiral
  Iter = "Iteration"              # The iteration number at which the item is found
  Age = "Age"                     # The age of the item
  Id = "ID"                       # The ID of the item
  C_lens = "Chain lengths"        # A list of the chain lengths in the item
  Max_C = "Longest chain length"  # The length of the longest chain in the item
  Contains = "Contains"           # The monomers that the polymer contains


  for r in self.get_reactables():
    item = r.get_sequence()
    age = r.get_age()
    # Translate
    sequence = self.standard_form(item)

    #Is it in the lookup table?
    if sequence not in self.get_lookup():

      # If not, check what kind of data to gether
      if len(item) == 1: # It's a monomer
        # Get the info
        lr = self.count_LR(item)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Monomer', 
                                        Length:1, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        Sequence:sequence})
      elif len(item) > 1: # It's a polymer
        # Get basic info
        length = len(item)
        lr = self.count_LR(item)
        bonds = self.homochiral_bond_counts(item)
        total_homos = bonds[0]+bonds[1]
        signed_ee = (lr[0]-lr[1])/(lr[0]+lr[1])
        homochirality = total_homos/(length-1)
        lhomochirality = bonds[0]/(length-1)
        rhomochirality = bonds[1]/(length-1)
        chains = self.chain_lengths(item)
        max_c = max(chains)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Polymer', 
                                        Length: length, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        nLhomo:bonds[0],
                                        nRhomo:bonds[1],
                                        nHomo:total_homos,
                                        sEE:signed_ee,
                                        pcHomo:homochirality,
                                        pcLhomo:lhomochirality,
                                        pcRhomo:rhomochirality,
                                        Sequence:sequence,
                                        C_lens:chains,
                                        Max_C:max_c})
      else:
        raise ValueError("There's something with length 0 in your history.")

    # Now that the data is searchable...
    # Log it
    new_log = self.get_lookup().get(sequence)
    new_log[Iter] = iteration
    new_log[Age] = age
    new_log[Id] = r.get_id()
    if len(item) > 1:
      new_log[Contains] = self.get_contents(r)
    self.add_log(new_log)


def parse_history(self):
  """ Create an array of plottable information from the history log. """
  parsed_hist = pd.DataFrame(self.get_history())
  self.set_hist_stats(parsed_hist)


In [39]:
#@title Low level info functions
%%add_to Reactables

## New ones ##

  
# Functions for getting the info we want to log
    

def get_contents(self,polymer):
  monomers = polymer.get_monomers()
  return [m.get_id() for m in monomers]


# Number of right and left monomers in a polymer (works on monomers too)


def count_LR(self, log):
    """ Return (n_Lefts, n_Rights) """
    if L and not R:
      return (sum(log), len(log)-sum(log))
    elif R and not L:
      return (len(log)-sum(log), sum(log))
      #unsure what to do here
        #raise ValueError("Your L's and R's are screwed up somehow.")
    


# HOMOCHIRALITY SEQUENCES #

def homochirality_sequence(self, log):
  """ Return a boolean list of bonds within a given logged polymer, True if the bond is
      homochiral. """
  return [i[0]==i[1] for i in list(more_itertools.pairwise(log))]

# def _is_L_True(self, log):
#     return log[:-1]


# def T_homochirality_sequence(self,h_seq,L_seq):
#     """ Return a boolean list of bonds within a polymer, true if the bond is 
#     a homochiral bond between two True monomers. Parameters are the outputs of 
#     homochirality_sequence() and _is_L_True(). """
#     return eAnd(h_seq,L_seq)

# HOMOCHIRAL BOND COUNTS #
    

def homochiral_bond_counts(self,log):
    """ Return (number of left homochiral bonds, number of right homochiral bonds) """
    homo = self.homochirality_sequence(log)
    west_true = log[:-1]
    true_homochiral = eAnd(homo,west_true)
    west_false = [not m for m in west_true]
    false_homochiral = eAnd(homo,west_false)
    if L and not R:
        return (sum(true_homochiral), sum(false_homochiral))
    elif R and not L: 
        return (sum(false_homochiral), sum(true_homochiral))
    else:
        raise ValueError("Your L's and R's are screwed up somehow.")


def chain_lengths(self,polylog):
  """ Return the lengths of the homochiral chains in a given polymer log. """
  count = 1
  lengths = [] 
  for n in range(1,len(polylog)):
    if polylog[n] == polylog[n-1]:
      count += 1
    else:
      lengths.append(count)
      count = 1
  lengths.append(count)
  return lengths
    

def standard_form(self,poly):
  if len(poly) == 0:
    return ''
  return ''.join(['L' if m else 'R' for m in poly])
    



## Old ones ##

# def hist_get_polymer_homochirality_of_bonds(self,polylog):
#   """ Return a boolean list of bonds within a given logged polymer, True if the bond is
#       homochiral. """
#   return [i[0]==i[1] for i in list(more_itertools.pairwise(polylog))]

# def hist_get_polymer_ee(self,polylog):
#   """ Return the (signed) enantiomeric excess of a logged polymer. """
#   length = len(polylog)
#   n_True = sum(polylog)
#   n_False = length - n_True
#   return (n_True - n_False) / length

# def hist_get_polymers(self,iteration):
#   """ Return a list of the polymer logs in an iteration. This is just the iteration
#   but without the monomers. """
#   return [r for r in iteration if len(r) > 1]

# def hist_count_longest_homochiral_chain(self,polylog):
#   """ Return the length of the longest homochiral chain given the log of a polymer. """
#   previous = None
#   count = 1
#   longest = 1
#   for monomer in polylog:
#     if monomer == previous:
#       count += 1
#     else:
#       longest = max(count, longest)
#       count = 1
#     previous = monomer
#   longest = max(count, longest)
#   return longest

# def hist_get_polymer_chain_lengths(self,polylog):
#   """ Return the lengths of the homochiral chains in a given polymer log. """
#   count = 1
#   lengths = [] 
#   for n in range(1,len(polylog)):
#     if polylog[n] == polylog[n-1]:
#       count += 1
#     else:
#       lengths.append(count)
#       count = 1
#   lengths.append(count)
#   return lengths

# def hist_get_iteration_chain_lengths(self,iteration):
#   """ Return the lengths of all the homochiral chains in polymers in a given iteration. """
#   polymers = self.hist_get_polymers(iteration)
#   chain_lengths = []
#   for polymer in polymers:
#     chain_lengths.extend(self.hist_get_polymer_chain_lengths(polymer))
#   return chain_lengths

# # Other new ones #

# def hist_get_leftright_homochiral_count_iteration(self,iteration):
#   """
#   returns a tuple (l,r)of the total number of left homochiral bonds and right homochiral bonds in an iteration 
#   """
#   #initilize l to keep track of the number of homochiral bonds
#   l=0
#   #initilize r to keep track of the number of homochiral bonds
#   r=0

#   #run through reactables in the iteration
#   for reactable in iteration:
#      #add the left homochiral bond count in each reactable
#      l += self.hist_get_leftright_homochiral_count_polymer(reactable)[0]
#      #add the right homochiral bond count in each reactable
#      r += self.hist_get_leftright_homochiral_count_polymer(reactable)[1]

#   #place l and r in a tuple
#   leftrightcount=(l,r)

#   #return the tuple
#   return leftrightcount

# def hist_get_leftright_homochiral_count_polymer(self,polymer):
#   """
#   returns a tuple (l,r)of the total number of left homochiral bonds and right homochiral bonds in an polymer 
#   """

#   #initialize l to keep track of the number of left homochiral bonds
#   l=0
#   #initialize r to keep trac of the number of right homochiral bonds
#   r=0

#   #go through the monomers in the polymer
#   for monomer in range(len(polymer)-1):

#       #check if they are homochiral
#         if (polymer[monomer]== polymer[monomer+1]):
#           #if so check if they are homochiral left
#           if (polymer[monomer]):
#             #if so add one to l
#             l += 1
#           #otherwise check if they are homochiral right
#           elif (not polymer[monomer]):
#             #if so add one to r
#             r += 1
#   #place l and r in a tuple
#   leftrightcount= (l,r)
#   #return the tuple
#   return leftrightcount

# def hist_get_bond_status(self,polylog):
#   """ Iterate through the monomers in polylog ONCE to get number of left homochiral
#   and right homochiral bonds, as well as chain lengths if at all possible. """
#   for m in polylog:
#     pass


# Dashboard

In [41]:
#@title Controls
# Initialize widgets for use in the dashboard
# Use a box to relate label and slider, just for space reasons.

# General parameters
bbbp = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
bbbp_label = ipywidgets.Label('BASE_BOND_BREAK_PROBABILITY:')
bbbp_box = ipywidgets.VBox([bbbp_label,bbbp])

lf = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
lf_label = ipywidgets.Label('LENGTH_FACTOR')
lf_box = ipywidgets.VBox([lf_label,lf])

n_val = ipywidgets.IntSlider(value=40, min=1, max=100, step=1)
n_val_label = ipywidgets.Label('N')
n_val_box = ipywidgets.VBox([n_val_label,n_val])

# Monomer parameters
pc = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
pc_label = ipywidgets.Label('POOF_CHANCE')
pc_box = ipywidgets.VBox([pc_label,pc])

bp = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
bp_label = ipywidgets.Label('BOND_PROB')
bp_box = ipywidgets.VBox([bp_label,bp])

# Simulation parameters
ps = ipywidgets.IntSlider(value=10, min=1, max=1000, step=1)
ps_label = ipywidgets.Label('POOL_SIZE')
ps_box = ipywidgets.VBox([ps_label,ps])

iterns = ipywidgets.IntSlider(value=10, min=1, max=1000, step=1)
iterns_label = ipywidgets.Label('ITERATIONS')
iterns_box = ipywidgets.VBox([iterns_label,iterns])

fusion = ipywidgets.Checkbox(description = "Check this box to allow fusion")
fusion_label = ipywidgets.Label('FUSION_FACTOR')
fusion_box = ipywidgets.VBox([fusion_label,fusion])

# Method selector
method_picker = ipywidgets.Dropdown(options=['bias','standard','poisson'],value="standard")
method_picker_label = ipywidgets.Label("Select calculation method:")
method_picker_box = ipywidgets.VBox([method_picker_label,method_picker])

# Parameters for Standard method
hbf = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbf_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR:')
hbf_box = ipywidgets.VBox([hbf_label,hbf])

hnif = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnif_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR:')
hnif_box = ipywidgets.VBox([hnif_label,hnif])

# Parameters for Bias method
hbfleft = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbfleft_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR_LEFT:')
hbfleft_box = ipywidgets.VBox([hbfleft_label,hbfleft])

hbfright = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbfright_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR_RIGHT')
hbfright_box = ipywidgets.VBox([hbfright_label,hbfright])

hnifleft = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnifleft_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT')
hnifleft_box = ipywidgets.VBox([hnifleft_label,hnifleft])

hnifright = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnifright_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT')
hnifright_box = ipywidgets.VBox([hnifright_label,hnifright])

# Parameters for Poisson method
lamb = ipywidgets.IntSlider(value=6, min=1, max=10, step=1)
lamb_label = ipywidgets.Label('LAMBDA')
lamb_box = ipywidgets.VBox([lamb_label,lamb])

pf = ipywidgets.FloatSlider(value=1.3, min=1, max=2, step=0.1)
pf_label = ipywidgets.Label('POISSON_FACTOR')
pf_box = ipywidgets.VBox([pf_label,pf])

#Parameters for monomer refill options
limitoptions = ipywidgets.Dropdown(options= ['do not refill monomers','refill monomers so that # of reactants = pool size','add random number of monomers','add a specified number of monomers',
'add a specified percent of the difference',
'add a decreasing percent of the difference starting at:','add a decreasing number of monomers starting at:'], value = 'refill monomers so that # of reactants = pool size')
limitoptions_label = ipywidgets.Label("if # of reactants drops below pool size...")
limitoptions_box = ipywidgets.VBox([limitoptions_label, limitoptions])

percentselection = ipywidgets.FloatSlider(min = 0, max = 1, step = 0.01)
percentselection_label = ipywidgets.Label('Specify percent')
percentselection_box = ipywidgets.VBox([percentselection_label, percentselection])

numberselection = ipywidgets.IntSlider(min = 0, max = 1000, step = 1)
numberselection_label = ipywidgets.Label('Specify number')
numberselection_box = ipywidgets.VBox([numberselection_label, numberselection])

na = ipywidgets.Label("|-----------------N/A-----------------|")
na_box = ipywidgets.VBox([na])


# An output box where we can print feedback (running, done, etc)
feedback = ipywidgets.Output()

# Buttons
# A button to run the simulation
button = ipywidgets.Button(description="GO!", button_style='primary')
# A button to export the results
export_button = ipywidgets.Button(description="Export data")
# A button to mark interesting results
star = ipywidgets.Checkbox(description="Mark interesting")

# Make labels for the different main groups of widgets
glob_title=ipywidgets.Label("General parameters")
mon_title=ipywidgets.Label("Monomer parameters")
sim_title=ipywidgets.Label("Simulation paramaters")

# Make boxes to contain the main widget groups, and populate them
# General parameters
globs = ipywidgets.VBox([glob_title,bbbp_box,lf_box,n_val_box])
# Monomer parameters
mons = ipywidgets.VBox([mon_title,pc_box,bp_box])
# Simulation parameters
sims = ipywidgets.VBox([sim_title,ps_box,iterns_box, fusion_box, limitoptions_box])

# Make boxes for the different kinds of calculation methods
bias_widgets = [hbfleft_box,hbfright_box,hnifleft_box,hnifright_box]
standard_widgets = [hbf_box,hnif_box]
poisson_widgets = [lamb_box, pf_box]

# Make boxes to handle the dynamic widget switching
dep_widgets = ipywidgets.VBox(standard_widgets)
advanced_widgets = ipywidgets.VBox([])

# Make boxes for layout
left_col = ipywidgets.VBox([mons,sims, advanced_widgets])
center_col = ipywidgets.VBox([globs,ipywidgets.HBox([button,star]),export_button,feedback])
right_col = ipywidgets.VBox([method_picker_box,dep_widgets])

# A name for the reactables bag
react_bag = None

def advancedoptions(change):
  """ 
  method that observes selections of the monomer refill dropdown menu 
  and changes the advanced_widgets box to displa the appropriate 
  advanced option
  """
  #if option selected needs a number specified
  if (change.new == "add a specified number of monomers" or change.new =="add a decreasing number of monomers starting at:"):
   #numberselection box displayed
    advanced_widgets.children = [numberselection_box]
  #if option selected needs a percent specified
  if (change.new =="add a specified percent of the difference" or change.new == "add a decreasing percent of the difference starting at:"):
    #the percentselection box is displayed
    advanced_widgets.children = [percentselection_box]
  #if option selected does not require further options
  if (change.new == "do not refill monomers" or  change.new == "refill monomers so that # of reactants = pool size" or change.new == "add random number of monomers"):
    #no advanced iwdgets are displayed
    advanced_widgets.children = []


def choose(change):
  """
  This function is magic. I don't know how it works.
  It's job is to handle switching out the widgets in the method selection. 
  """                 
  if change.new == "bias":
    dep_widgets.children = bias_widgets
  elif change.new == "standard":
    dep_widgets.children = standard_widgets
  elif change.new == "poisson":
    dep_widgets.children = poisson_widgets

def export(b):
  """
  Export the stats dataframe as a csv with the filename encoding the parameters 
  for the run.
  """
  # Store the parameters in a dictionary for exporting.
  params = {"BASE_BOND_BREAK_PROBABILITY":BASE_BOND_BREAK_PROBABILITY, 
          "HOMOCHIRAL_BREAK_FACTOR":HOMOCHIRAL_BREAK_FACTOR,
          "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR,
          "LENGTH_FACTOR":LENGTH_FACTOR,
          "N":N,
          "LAMBDA":LAMBDA,
          "HOMOCHIRAL_BREAK_FACTOR_LEFT":HOMOCHIRAL_BREAK_FACTOR_LEFT,
          "HOMOCHIRAL_BREAK_FACTOR_RIGHT":HOMOCHIRAL_BREAK_FACTOR_RIGHT,
          "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT,
          "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT,
          "POOF_CHANCE":POOF_CHANCE,
          "BOND_PROB":BOND_PROB,
          "POOL_SIZE":POOL_SIZE,
          "ITERATIONS":ITERATIONS,
          "METHOD":METHOD,
          "POISSON_FACTOR":POISSON_FACTOR,
          "FUSION":FUSION}
  # Want the timestamp created in the other function
  global timestamp
  # Create the mark, empty in case it's an uninteresting run
  mark = ""
  # Check that the react bag isn't empty
  if react_bag != None:
    # Mark the run if the interesting box is checked
    if star.value:
      mark = "_"
    # Export parameter file
    dict_to_csv(params,f"{wdir}/output/{mark}{timestamp}_params.csv")
    # Get results
    stats = react_bag.get_stats()
    # Export data file
    stats.to_csv(f"{wdir}/output/{mark}{timestamp}_data.csv")
    # Say "I did it"
    with feedback:
      clear_output()
      print(f"Data exported.\nInteresting = {star.value}")


def run_sim(b):
  """Method runs through a simulation"""

  #let the user know what is going on
  with feedback:
    clear_output()
    print("Running...")

  #initialize global parameters
  global BASE_BOND_BREAK_PROBABILITY
  global HOMOCHIRAL_BREAK_FACTOR
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR
  global LENGTH_FACTOR
  global N
  global LAMBDA
  global HOMOCHIRAL_BREAK_FACTOR_LEFT
  global HOMOCHIRAL_BREAK_FACTOR_RIGHT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT
  global POOF_CHANCE
  global BOND_PROB
  global POOL_SIZE
  global ITERATIONS
  global METHOD
  global break_prob_lookup_table
  global poisson_dict
  global react_bag
  global interesting
  global timestamp
  global POISSON_FACTOR
  global FUSION
  global REFILL
  global L
  global R
  global P
  global N_RANDS
  global rands

  #set monomer refill variables
  REFILL = ["no refill"]

  #set monomer refill variables
  if limitoptions.value == 'refill monomers so that # of reactants = pool size':
    REFILL = ["refill normal"]
  elif limitoptions.value == 'add a specified number of monomers':
    REFILL = ["refill number",numberselection.value]
  elif limitoptions.value == 'add a specified percent of the difference':
    REFILL = ["refill percent",percentselection.value]
  elif limitoptions.value == 'add random number of monomers':
    REFILL = ["refill random"]
  elif limitoptions.value == 'add a decreasing percent of the difference starting at:':
    REFILL = ["refill percent decrease",percentselection.value]
  elif limitoptions.value == 'add a decreasing number of monomers starting at:':
    REFILL = ["refill number decrease",numberselection.value]
  else:
    print("NO REFILL!!!!!")

  #set variables to values given by dashboard
  timestamp = int(np.ceil(time.time()))
  break_prob_lookup_table = {}
  poisson_dict={}
  N_RANDS = 1000
  rands = []
  L = True
  R = False
  P = None
  poisson_dict={}
  FUSION = fusion.value
  POISSON_FACTOR = pf.value
  BASE_BOND_BREAK_PROBABILITY=bbbp.value
  HOMOCHIRAL_BREAK_FACTOR=hbf.value
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR =hnif.value
  LENGTH_FACTOR=lf.value
  N = n_val.value
  LAMBDA = lamb.value
  HOMOCHIRAL_BREAK_FACTOR_LEFT=hbfleft.value
  HOMOCHIRAL_BREAK_FACTOR_RIGHT=hbfright.value
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT=hnifleft.value
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT=hnifright.value
  POOF_CHANCE=pc.value
  BOND_PROB=bp.value
  POOL_SIZE=ps.value
  ITERATIONS=iterns.value
  METHOD = method_picker.value

  #create initial reactables bag
  react_bag = Reactables(make_pool(POOL_SIZE))
  #run through the simulation
  react_bag.simulate(POOL_SIZE, ITERATIONS, METHOD)
  interesting = star.value
  
  #get the history from the reactables bag
  stats = react_bag.get_stats()

  #keep user updated on what program is doing
  with feedback:
    clear_output()
    print("Plotting...")

  ##################
  #PLOTTING METHODS#
  ##################

  with tb.output_to("Signed EE of Polymers by Iteration", select=True):
    #plots a scatter plot of signed ee of the polymers by Iteration 
    #clear the tab
    tb.clear_tab()
    #create a data frame
    df = stats
    #limit polymer that are recorded to being of length >=5
    df = df[df['Length'] >= 5]
    #record amounts of time identical data points occur
    counts = df.groupby("Iteration")['Signed ee'].value_counts().reset_index(name='count')
    #plot it!
    fig = px.scatter(counts, x="Iteration", y="Signed ee", size='count')
    fig.show()


  with tb.output_to("Proportion of Bond Types", select=False):
    #plot # of right, # of left, and # of heterochiral bonds in each iteration
    #clear the tab
    tb.clear_tab()
    #make dataframe
    df = stats
    #data for lefts
    lefts = df.groupby("Iteration")["#LeftHomochiral"].sum().rename("LL")
    #data for rights
    rights = df.groupby("Iteration")["#RightHomochiral"].sum().rename("RR")
    #data for heteros
    total = (df.groupby("Iteration")['Length'].sum() - df.groupby("Iteration")['Length'].count()).rename("Total")
    #make datafram
    bondcounts = pd.DataFrame([lefts, rights, total]).transpose()
    bondcounts["LR"] = bondcounts["Total"] - bondcounts["RR"] - bondcounts["LL"]
    bondcounts = bondcounts.apply(lambda x : x / bondcounts["Total"])
    
    fig = go.Figure()
    #plot it!!
    fig.add_trace(go.Scatter(y=bondcounts["LL"],
                        mode='lines',
                        name='left homochiral'))
    fig.add_trace(go.Scatter(y=bondcounts["RR"],
                        mode='lines',
                        name='right homochiral'))
    fig.add_trace(go.Scatter(y=bondcounts["LR"],
                       mode='lines', name='heterochiral'))
    fig.update_layout(title='Proportion of LL, RR, and LR bonds by iteration',
                    xaxis_title='Iteration',
                    yaxis_title='Proportion')
    fig.update_yaxes(range=[0, 1])

    fig.show()

  # with tb.output_to("Homochirality vs Length through Time", select=False):
  #   #Graph of homochirality vs length animated by iteration
  #   #clear the tab
  #   tb.clear_tab()
  #   #create data frame
  #   df = stats
  #   maxlen = df["Length"].max()
  #   #plot it!!
  #   fig = px.scatter(df, x="Length", y="%Homochirality",animation_frame="Iteration",
  #                   range_x=[-1,maxlen+1],range_y=[-0.01,1.2])
  #   fig.show()

  with tb.output_to("Homochirality vs Length", select=False):
    #plot homochirality vs length
    #clear the tab
    tb.clear_tab()
    #get the data
    df = stats
    #limit polymer that are recorded to being of length >=5
    df = df[df['Length'] >= 5]
    #make the dataframe
    df = df.groupby("Length")['%Homochirality'].value_counts().reset_index(name='count')
    #plot it!!
    fig = px.scatter(df, x="Length",y="%Homochirality",size='count')
    fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                    selector=dict(mode='markers'))
    fig.show()

  with tb.output_to("Homochirality vs Age", select=False):
    #plot Homochirality vs Age
    #clear the tab
    tb.clear_tab()
    #create dataframe
    df = stats
    #limit polymer that are recorded to being of length >=5
    df = df[df['Length'] >= 5]
    df = df.groupby("Age")['%Homochirality'].value_counts().reset_index(name='count')
    #plot it!
    fig = px.scatter(df, x="Age",y="%Homochirality",size='count')
    fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                    selector=dict(mode='markers'))
    fig.show()

  # with tb.output_to("Visualize Break Probs", select=False):
  #   #create a graph to visualize Break Probabilities of monoemers in various polymers
  #   #clear the tab
  #   tb.clear_tab()
  #   #create a left monomers
  #   a = Monomer(hand = True)
  #   #create a polymer b
  #   b = Polymer(monomers = [a])
  #  #create right monomer
  #   c = Monomer(hand = False)
  #   #add it do a polymer
  #   d = Polymer(monomers = [c])
  #   #create x to keep track of index of the monomer
  #   x = []
  #   #create y to keep track of break probs
  #   y = []
  #   #create method to keep track of the method used to calculate the brk prob
  #   method = []
  #   #create length to keep track of the homochiralchainlength
  #   length=[]
  #   for n in range(30):
  #     #loop through polymers up to length 30
  #     for z in range(b.get_length()-1):
  #       #loop through all of the monomers in the polymer
  #       #add normal break prob to y
  #       y.append(b.eastbondbreakprobability(z))
  #       #add method used
  #       method.append("Normal")
  #       #add index of monomers
  #       x.append(z+1)
  #       #add homochiral chainlength
  #       length.append(b.get_length())
  #       #repeat for poisson method
  #       y.append(b.poissonbreakprobability(z))
  #       method.append("poisson")
  #       length.append(b.get_length())
  #       x.append(z+1)
  #       #repeat for bias method with left polymer
  #       y.append(b.biaseastbondbreakprobability(z))
  #       method.append("left")
  #       length.append(b.get_length())
  #       x.append(z+1)
  #       #repeat for biad method with right polymer
  #       y.append(d.biaseastbondbreakprobability(z))
  #       method.append("right")
  #       x.append(z+1)
  #       length.append(b.get_length())
  #     #add a left momoner to b
  #     monomer = Monomer(hand = True)
  #     b.append(monomer)
  #     #add a right monomer to d
  #     monomer = Monomer(hand = False)
  #     d.append(monomer)
  #   #create data frame with lists we just compiled
  #   data = {'NumberMonomer':  x,
  #     'HomochiralChainLength': length, 'Breakprob': y, 'Method': method}
  #   df = pd.DataFrame(data, columns = ['NumberMonomer',
  #     'HomochiralChainLength', 'Breakprob', 'Method'])
  #   fig = go.Figure()
  #   #plot it!!
  #   fig = px.scatter(df, x="NumberMonomer", y="Breakprob", animation_frame="HomochiralChainLength",
  #         size = "HomochiralChainLength", color="Method", hover_name="Method", range_x=[0,30], range_y=[-0.2,1.2], title = "Brk Probs of Homochiral chain")
  #   fig.show()

  #   #create a plot of the effects of length on the basebond brk prob
  #   #create list x to keep track of lengths
  #   x = []
  #   #create list y to keep track of brk probs
  #   y = []
  #   for n in range(100):
  #     #loop through lengths up to 100
  #     #append length to x
  #     x.append(n)
  #     #append brk prob to y using length equation used in program
  #     y.append(BASE_BOND_BREAK_PROBABILITY * LENGTH_FACTOR**(n/N))
  #   #compile data into a dataframe
  #   data = {'Length':  x,'BrkProb': y}
  #   df = pd.DataFrame(data, columns = ['Length',
  #     'BrkProb'])
  #   #plot it!!
  #   fig = px.line(df, x = 'Length', y = 'BrkProb', title = "Effects of Length on Break Probs")
  #   fig.show()

  # with tb.output_to("Length by homochirality through age", select = False):
  #   #create a graph of length by homochirality animated by age
  #   #clear the tab
  #   tb.clear_tab()
  #   #gather data
  #   df = stats
  #   #plot it !!
  #   fig = px.scatter(df, x="Length", y="%Homochirality", animation_frame="Age", hover_name="%Homochirality", range_x=[-1,maxlen+1])
  #   fig.show()

  # with tb.output_to("Age by homochirality through length", select = False):
  #   #create a graph of age by homochirality animated by length
  #   #clear tab
  #   tb.clear_tab()
  #   #create dataframe
  #   df = stats
  #   #plot it!!1
  #   fig = px.scatter(df, x="Age", y="%Homochirality", animation_frame="Length", hover_name="%Homochirality")
  #   fig.show()

  # with tb.output_to("Length histogram over time", select=False):
  #   #create a histogram of length over time
  #   #clear tab
  #   tb.clear_tab()
  #   #gather data
  #   df = stats
  #   #plot it!!
  #   fig = px.histogram(df, x="Length", animation_frame="Iteration",range_x=(0,max(df["Length"])),
  #                      nbins=max(df["Length"]))
  #   fig.show()

  #keep user updated on what the program is doing
  with feedback:
    clear_output()
    print("Done!")

#call methods to observe dynamics of dashboard
#observe if run button is clicked
button.on_click(run_sim)

#observe if export button is clicked
export_button.on_click(export)

#create display container
container = ipywidgets.HBox([left_col,center_col,right_col])

#observe what is selected in monomer refil dropdown methods
limitoptions.observe(advancedoptions)

#observe which method is chosen
method_picker.observe(choose, names="value")

#display the container
display(container)

#tabs to be displayed
plots = ["Signed EE of Polymers by Iteration","Proportion of Bond Types",
          "Homochirality vs Length","Homochirality vs Age"]
#create tabs
tb = widgets.TabBar(plots)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<bound method get_handedness of False-monomer>
<bound method get_handedness of False-monomer>


TypeError: ignored

# Batch engine

In [40]:
#@title Batch functionality

#make widgets for folder and batch names
folder_label = ipywidgets.Label("Name the folder you want to export to:")
batch_name_label = ipywidgets.Label("Name your batch:")
folder_widget = ipywidgets.Text(value = "small_tests")
batch_name_widget = ipywidgets.Text(value = "default")
folder_hbox = ipywidgets.HBox([folder_label,folder_widget])
batch_name_hbox = ipywidgets.HBox([batch_name_label,batch_name_widget])
# make widgets for reps
repswidget = ipywidgets.IntSlider(min = 1, max = 6)
repslabel = ipywidgets.Label("Select how many times you want each set of parameter values to run")
reps_hbox = ipywidgets.HBox([repslabel,repswidget])

#General parameters

#for each parameter...

#make a check box(to indicate whether or not variable will be dynamic)
checkbbbp = ipywidgets. Checkbox(description = "BASE_BOND_BREAK_PROBABILITY")
#make a widget to select a static value
staticvaluebbbp = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.50)
#make a widget to select a range of values for dynamics
rangevaluebbbp = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
#make a widget to select a steo value for the ranges
stepvaluebbbp = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01, value = .1)
#box to hold widgets that will change when parameter is dynamic/static
bbbpdep_widgets = ipywidgets.HBox([staticvaluebbbp,na,na])
#make a box to hold all the widgets for the parameter you would like displayed
bbbp_hbox = ipywidgets.HBox([checkbbbp,bbbpdep_widgets])
#repeat for all other parameters

checklf = ipywidgets.Checkbox(description = "LENGTH_FACTOR")
rangevaluelf = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.1)
staticvaluelf = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.6)
stepvaluelf = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01, value = .1)
lfdep_widgets = ipywidgets.HBox([staticvaluelf,na,na])
lf_hbox = ipywidgets.HBox([checklf,lfdep_widgets])


checkn_val = ipywidgets.Checkbox(description = "N")
staticvaluen_val = ipywidgets.IntSlider(min=0,max = 100, step = 1, value = 40)
rangevaluen_val = ipywidgets.IntRangeSlider(min=0,max = 100, step = 1)
stepvaluen_val = ipywidgets.IntSlider(min=1,max = 100, value = 1)
n_valdep_widgets = ipywidgets.HBox([staticvaluen_val,na,na])
n_val_hbox = ipywidgets.HBox([checkn_val,n_valdep_widgets])

#make a label for general parameters
generalparameters_label = ipywidgets.Label("General Parameters")

# Monomer parameters
checkpc = ipywidgets.Checkbox(description = "POOF_CHANCE")
staticvaluepc = ipywidgets.FloatSlider(min = 0, max =1 , step =0.01, value = 0.33)
rangevaluepc = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluepc = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01, value = 0.1)
pcdep_widgets = ipywidgets.HBox([staticvaluepc,na,na])
pc_hbox = ipywidgets.HBox([checkpc,pcdep_widgets])

checkbp = ipywidgets.Checkbox(description = "BOND_PROB")
staticvaluebp = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.33)
rangevaluebp = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluebp = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01, value = 0.1)
bpdep_widgets = ipywidgets.HBox([staticvaluebp,na,na])
bp_hbox = ipywidgets.HBox([checkbp,bpdep_widgets])

#make a label for monomer parameters
monomerparameters_label = ipywidgets.Label("Monomer Parameters")

# Simulation parameters
checkps = ipywidgets.Checkbox(description = "POOL_SIZE")
staticvalueps = ipywidgets.IntSlider(min=1, max = 1000, step = 1, value = 100)
rangevalueps = ipywidgets.IntRangeSlider(min=1, max = 1000, step = 1)
stepvalueps = ipywidgets.IntSlider(min=1, max = 1000, step = 1, value = 1)
psdep_widgets = ipywidgets.HBox([staticvalueps,na,na])
ps_hbox = ipywidgets.HBox([checkps,psdep_widgets])

checkiterns = ipywidgets.Checkbox(description = "ITERATIONS")
staticvalueiterns= ipywidgets.IntSlider(min=1, max = 1000, step = 1, value = 100)
rangevalueiterns= ipywidgets.IntRangeSlider(min=1, max = 1000, step = 1)
stepvalueiterns= ipywidgets.IntSlider(min=1, max = 1000, step = 1, value = 1)
iternsdep_widgets = ipywidgets.HBox([staticvalueiterns,na,na])
iterns_hbox = ipywidgets.HBox([checkiterns,iternsdep_widgets])

#make label for simulation parameters
simulationparameters_label = ipywidgets.Label("Simulation Parameters")

#This is where widgets become a little different...

# Method selector
#only need a dropdown menu- types of methods can not be dynamic
staticvaluemethod_picker= ipywidgets.Dropdown(options= ["Standard", "Poisson","Bias"])
#create a label to explain what dropdown presents
method_picker_label = ipywidgets.Label("First pick which calculation method you want to focus on:")
#create a box for method picker widgets
method_picker_hbox = ipywidgets.HBox([method_picker_label, staticvaluemethod_picker])

#now we have to make the parameters for all the different calculation methods

# Parameters for Standard method
#these are defined the same as parameters before, check box, static,range,step,dep_widgets, and box
checkhbf = ipywidgets.Checkbox(description = "HOMOCHIRAL_BREAK_FACTOR")
staticvaluehbf = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehbf = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehbf = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01)
hbfdep_widgets = ipywidgets.HBox([staticvaluehbf,na,na])
hbf_hbox = ipywidgets.HBox([checkhbf,hbfdep_widgets])

checkhnif = ipywidgets.Checkbox(description = "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR")
staticvaluehnif = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehnif = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehnif = ipywidgets.FloatSlider(min=0.01,max = 1.0, step = 0.01)
hnifdep_widgets = ipywidgets.HBox([staticvaluehnif,na,na])
hnif_hbox = ipywidgets.HBox([checkhnif,hnifdep_widgets])

#label for standard method parameters
standardmethodparameters_label = ipywidgets.Label("Standard Method Parameters")
#box to display standard method parameters
standard_vbox =[standardmethodparameters_label,hbf_hbox,hnif_hbox]
#repeat for bias and poisson method parameters

# Parameters for Bias method
checkhbfleft = ipywidgets.Checkbox(description = "HOMOCHIRAL_BREAK_FACTOR_LEFT")
staticvaluehbfleft = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehbfleft = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehbfleft = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01)
hbfleftdep_widgets = ipywidgets.HBox([staticvaluehbfleft,na,na])
hbfleft_hbox = ipywidgets.HBox([checkhbfleft,hbfleftdep_widgets])

checkhbfright = ipywidgets.Checkbox(description= "HOMOCHIRAL_BREAK_FACTOR_LEFT")
staticvaluehbfright = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehbfright = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehbfright = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01)
hbfrightdep_widgets = ipywidgets.HBox([staticvaluehbfright,na,na])
hbfright_hbox = ipywidgets.HBox([checkhbfright,hbfrightdep_widgets])

checkhnifleft = ipywidgets.Checkbox(description = "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT")
staticvaluehnifleft = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehnifleft = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehnifleft = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01)
hnifleftdep_widgets = ipywidgets.HBox([staticvaluehnifleft,na,na])
hnifleft_hbox = ipywidgets.HBox([checkhnifleft,hnifleftdep_widgets])

checkhnifright = ipywidgets.Checkbox(description = "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT")
staticvaluehnifright = ipywidgets.FloatSlider(min=0,max = 1.0, step = 0.01, value = 0.5)
rangevaluehnifright = ipywidgets.FloatRangeSlider(min=0,max = 1.0, step = 0.01)
stepvaluehnifright = ipywidgets.FloatSlider(min=0.1,max = 1.0, step = 0.01)
hnifrightdep_widgets = ipywidgets.HBox([staticvaluehnifright,na,na])
hnifright_hbox = ipywidgets.HBox([checkhnifright,hnifrightdep_widgets])

biasmethodparameters_label = ipywidgets.Label("Bias Method Parameters")
bias_vbox = [biasmethodparameters_label,hbfleft_hbox,hbfright_hbox,hnifleft_hbox,hnifright_hbox]

#parameters for Poisson factor
checkpf = ipywidgets.Checkbox(description = "POISSON_FACTOR")
staticvaluepf = ipywidgets.FloatSlider(min=1,max = 2, step = 0.1, value = 1.3)
rangevaluepf = ipywidgets.FloatRangeSlider(min=1,max = 2, step = 0.01)
stepvaluepf = ipywidgets.FloatSlider(min=0.1,max = 1, step = .01)
pfdep_widgets = ipywidgets.HBox([staticvaluepf,na,na])
pf_hbox = ipywidgets.HBox([checkpf,pfdep_widgets])


checklamb = ipywidgets.Checkbox(description = "LAMBDA")
staticvaluelamb = ipywidgets.IntSlider(min=0,max = 10, value = 6)
rangevaluelamb = ipywidgets.IntRangeSlider(min=0,max = 10)
stepvaluelamb = ipywidgets.IntSlider(min=1,max = 10)
lambdep_widgets = ipywidgets.HBox([staticvaluelamb,na,na])
lamb_hbox = ipywidgets.HBox([checklamb,lambdep_widgets])


poissonmethodparameters_label = ipywidgets.Label("Poisson Method Parameters")
poisson_vbox = [poissonmethodparameters_label,lamb_hbox, pf_hbox]

#widget for display of selected method parameters
methodselectwidgets = ipywidgets.VBox(standard_vbox)


#make label for parameters that are determined by boolean values: a little different than normal
boolean_label = ipywidgets.Label("Boolean Parameters")

#Fusion widgets are booleans

#still make a checkbox to indicate wheter or not it will be dynamic
checkfusion = ipywidgets.Checkbox(description = "FUSION_FACTOR")
#static value = a drop down to select which value you want
staticvaluefusion= ipywidgets.Dropdown(options= [("on", True), ("off",False)])
#rather than step and range, two check boxes for both options
polymerfusionon = ipywidgets.Checkbox(description= "Polymer Fusion On")
polymerfusionoff = ipywidgets.Checkbox(description = "PolymerFusion Off")
#make a vertical box to display the two checkboxes as a dynamic option
fusion_Vbox= ipywidgets.VBox([polymerfusionon,polymerfusionoff])
#make a box to display either static or dynamic widgets
fusiondep_widgets = ipywidgets.HBox([staticvaluefusion,na])
#mea a box to display fusion widgets
fusion_hbox = ipywidgets.HBox([checkfusion,fusiondep_widgets])

#Next are widgets for parameters to determine how reactant pool will refill(monomerrefill)

#initial monomerrefill choice widgets
#only a dropdown - refill methods are not dynamic
monomerrefill = ipywidgets.Dropdown(options = ['do not refill monomers','refill monomers so that # of reactants = pool size','add random number of monomers','add a specified number of monomers',
'add a specified percent of the difference',
'add a decreasing percent of the difference starting at:','add a decreasing number of monomers starting at:'], value = 'refill monomers so that # of reactants = pool size')
#make a label explaining what the dropdown menu indicates
monomerrefilllabel = ipywidgets.Label("When reactables drop below poolsize...")
#make a box to show refill options
monomerrefillbox = ipywidgets.VBox([monomerrefilllabel,monomerrefill])

#monomer refill check boxes for all different additional parameters(to make dynamic or static)
checkmonomernochoice = ipywidgets.Label('|-------------------N/A: no additional paramters----------------------------------------|')
checkmonomerpercent = ipywidgets.Checkbox(description = "Specified percent")
checkmonomernumber = ipywidgets.Checkbox(description = "Specified number")

#creation of percent selecter(for static, range, and step)
percentslider = ipywidgets.FloatSlider(min = 0, max = 1, step = 0.01)
percentrange = ipywidgets.FloatRangeSlider(min = 0,max = 1,step =0.01)
percentstep = ipywidgets.FloatSlider(min = 0.1, max = 1, step = 0.01)

#creation of number selecter(for static, range, and step)
numberslider = ipywidgets.IntSlider(min = 0, max = 1000, step = 1)
numberrange = ipywidgets.IntRangeSlider(min =0, max = 1000,step = 1)
numberstep = ipywidgets.IntSlider(min = 1, max = 1000, step = 1)

#box of widgets for static values for all monomer refill options
staticvaluenochoice_hbox = ipywidgets.HBox([na,na,na])
staticvaluepercent_hbox = ipywidgets.HBox([percentslider,na,na])
staticvaluenumber_hbox = ipywidgets.HBox([numberslider,na,na])

#make boxes for dynamic options for all widgets
rangevaluepercent_hbox = ipywidgets.HBox([na, percentrange,percentstep])
rangevaluenumber_hbox = ipywidgets.HBox([na, numberrange,numberstep])

#labels for all the different types of monomer refill methods
refillnormallabel = ipywidgets.Label("Refill Method Parameters: Option selected: Refill monomers so poolsize is always =< reactables bag")
refillrandomlabel = ipywidgets.Label("Refill Method Parameters: Option selected: Randomly refill monomers")
norefilllabel = ipywidgets.Label("Refill Method Parameters: Option selected: do not refill monomers")

refillpercentlabel = ipywidgets.Label("Refill Method Parameters: Option selected :refill a specified percent of monomers")
refillpercentdecreaselabel = ipywidgets.Label("Refill Method Parameters: Option selected: refill decreasing percent of monomers starting at a specified percent") 

refillnumberlabel = ipywidgets.Label("Refill Method Parameters: Option selected: refill a specified number of monomers")
refillnumberdecreaselabel = ipywidgets.Label("Refill Method Parameters: Option selected: refill decreasing number of monomers starting at a specified number") 

#initialize label as refillnormal
monomer_label = ipywidgets.VBox([refillnormallabel])

#initializing widgets for nochoice, percent, and number choice as static
nochoicedep_widgets = ipywidgets.HBox([checkmonomernochoice,staticvaluenochoice_hbox])
percentchoicedep_widgets = ipywidgets.HBox([checkmonomerpercent,staticvaluepercent_hbox])
numberchoicedep_widgets = ipywidgets.HBox([checkmonomernumber,staticvaluenumber_hbox])

#initializing monomer refill dep widgets in nochoice dep widgets
monomerrefilldep_widgets= ipywidgets.HBox([nochoicedep_widgets])


#display of monomer refill box
monomerrefill_vbox = ipywidgets.VBox([monomer_label, monomerrefilldep_widgets])

#
#a go button to click when all variables are selected
gobutton = ipywidgets.Button(description="GO!", button_style='primary')


#Headers for dashboard
checkbox_col_label = ipywidgets.Label("Check variables you want to be dynamic:----------------- |")
static_col_label =ipywidgets.Label("Select Value:------------------- |")
range_col_label = ipywidgets.Label("Seclect Range:  -----------  |")
step_col_label = ipywidgets.Label("Select Step: (ex. Range of [0-5] and step =1 tests:[0,1,2,3,4])")

#box for header
header_hbox = ipywidgets.HBox([checkbox_col_label,static_col_label,range_col_label,step_col_label])

#header for boolean parameters(different instructions)
booleanheader = ipywidgets.Label("Check variables you want to be dynamic:------- |Select Value:------------------------- |Check all conditions you would like to evaluate:")


#creation of a button to ask user if they want to continue
continuebutton = ipywidgets.Button(description = 'Continue',button_style = 'primary')
continue_box = ipywidgets.VBox([])

#creation of dashboard: with all those darn boxes
container = ipywidgets.VBox([folder_hbox, batch_name_hbox, reps_hbox, method_picker_hbox, monomerrefillbox,header_hbox, generalparameters_label, bbbp_hbox,lf_hbox,n_val_hbox,monomerparameters_label,
      pc_hbox ,bp_hbox,simulationparameters_label,monomerrefill_vbox,ps_hbox ,iterns_hbox ,methodselectwidgets,boolean_label,booleanheader,fusion_hbox, gobutton])


#Methods for dynamics


def changebbbp(change):
  if change.new == True:
    bbbpdep_widgets.children = ([na,rangevaluebbbp,stepvaluebbbp])
  if change.new == False:
    bbbpdep_widgets.children = ([staticvaluebbbp,na,na])

checkbbbp.observe(changebbbp)

def changelf(change):
  if change.new == True:
    lfdep_widgets.children = ([na,rangevaluelf,stepvaluelf])
  if change.new == False:
    lfdep_widgets.children = ([staticvaluelf,na,na])

checklf.observe(changelf)

def changen_val(change):
  if change.new == True:
    n_valdep_widgets.children = ([na,rangevaluen_val,stepvaluen_val])
  if change.new == False:
    n_valdep_widgets.children = ([staticvaluen_val,na,na])

checkn_val.observe(changen_val)


def changepc(change):
  if change.new == True:
    pcdep_widgets.children = ([na,rangevaluepc,stepvaluepc])
  if change.new == False:
    pcdep_widgets.children = ([staticvaluepc,na,na])

checkpc.observe(changepc)

def changebp(change):
  if change.new == True:
    bpdep_widgets.children = ([na,rangevaluebp,stepvaluebp])
  if change.new == False:
    bpdep_widgets.children = ([staticvaluebp,na,na])

checkbp.observe(changebp)

def changeps(change):
  if change.new == True:
    psdep_widgets.children = ([na,rangevalueps,stepvalueps])
  if change.new == False:
    psdep_widgets.children = ([staticvalueps,na,na])

checkps.observe(changeps)

def changeiterns(change):
  if change.new == True:
    iternsdep_widgets.children = ([na,rangevalueiterns,stepvalueiterns])
  if change.new == False:
    iternsdep_widgets.children = ([staticvalueiterns,na,na])

checkiterns.observe(changeiterns)

def changefusion(change):
  if change.new == True:
    fusiondep_widgets.children = ([na,fusion_Vbox])
  if change.new == False:
    fusiondep_widgets.children = ([staticvaluefusion,na])

checkfusion.observe(changefusion)

def changehbf(change):
  if change.new == True:
    hbfdep_widgets.children = ([na,rangevaluehbf,stepvaluehbf])
  if change.new == False:
    hbfdep_widgets.children = ([staticvaluehbf,na,na])

checkhbf.observe(changehbf)

def changehnif(change):
  if change.new == True:
    hnifdep_widgets.children = ([na,rangevaluehnif,stepvaluehnif])
  if change.new == False:
    hnifdep_widgets.children = ([staticvaluehnif,na,na])

checkhnif.observe(changehnif)

def changehnifleft(change):
  if change.new == True:
    hnifleftdep_widgets.children = ([na,rangevaluehnifleft,stepvaluehnifleft])
  if change.new == False:
    hnifleftdep_widgets.children = ([staticvaluehnifleft,na,na])

checkhnifleft.observe(changehnifleft)

def changehnifright(change):
  if change.new == True:
    hnifrightdep_widgets.children = ([na,rangevaluehnifright,stepvaluehnifright])
  if change.new == False:
    hnifrightdep_widgets.children = ([staticvaluehnifright,na,na])

checkhnifright.observe(changehnifright)

def changehbfleft(change):
  if change.new == True:
    hbfleftdep_widgets.children = ([na,rangevaluehbfleft,stepvaluehbfleft])
  if change.new == False:
    hbfleftdep_widgets.children = ([staticvaluehbfleft,na,na])

checkhbfleft.observe(changehbfleft)

def changehbfright(change):
  if change.new == True:
    hbfrightdep_widgets.children = ([na,rangevaluehbfright,stepvaluehbfright])
  if change.new == False:
    hbfrightdep_widgets.children = ([staticvaluehbfright,na,na])

checkhbfright.observe(changehbfright)

#parameters for Poisson factor
def changepf(change):
  if change.new == True:
    pfdep_widgets.children = ([na,rangevaluepf,stepvaluepf])
  if change.new == False:
    pfdep_widgets.children = ([staticvaluepf,na,na])

checkpf.observe(changepf)

def changelamb(change):
  if change.new == True:
    lambdep_widgets.children = ([na,rangevaluelamb,stepvaluelamb])
  if change.new == False:
    lambdep_widgets.children = ([staticvaluelamb,na,na])

checklamb.observe(changelamb)


def changemethod_picker(change):
  if change.new == "Standard":
    methodselectwidgets.children = (standard_vbox)
  if change.new == "Poisson":
    methodselectwidgets.children = (poisson_vbox)
  if change.new == "Bias":
    methodselectwidgets.children = (bias_vbox)


staticvaluemethod_picker.observe(changemethod_picker, names = "value")


def changpercentrefill(change):
  if change.new == True:
    percentchoicedep_widgets.children = ([checkmonomerpercent, rangevaluepercent_hbox])
  if change.new == False:
    percentchoicedep_widgets.children = ([checkmonomerpercent, staticvaluepercent_hbox])
checkmonomerpercent.observe(changpercentrefill, names = "value")

def changenumberrefill(change):
  if change.new == True:
    numberchoicedep_widgets.children = ([checkmonomernumber,rangevaluenumber_hbox])
  if change.new == False:
    numberchoicedep_widgets.children = ([checkmonomernumber, staticvaluenumber_hbox])
checkmonomernumber.observe(changenumberrefill, names = "value")

def changemonomerstatus(change):
  if change.new == 'do not refill monomers':
    monomerrefilldep_widgets.children = ([nochoicedep_widgets])
    monomer_label.children = ([norefilllabel])
  if change.new == 'refill monomers so that # of reactants = pool size':
    monomerrefilldep_widgets.children = ([nochoicedep_widgets])
    monomer_label.children = ([refillnormallabel])
  if change.new == 'add random number of monomers':
    monomerrefilldep_widgets.children = ([nochoicedep_widgets])
    monomer_label.children = ([refillrandomlabel])
  if change.new == 'add a specified number of monomers':
    monomerrefilldep_widgets.children = ([numberchoicedep_widgets])
    monomer_label.children = ([refillnumberlabel])
  if change.new == 'add a specified percent of the difference':
    monomerrefilldep_widgets.children = ([percentchoicedep_widgets])
    monomer_label.children = ([refillpercentlabel])
  if change.new == 'add a decreasing percent of the difference starting at:':
    monomerrefilldep_widgets.children = ([percentchoicedep_widgets])
    monomer_label.children = ([refillpercentdecreaselabel])
  if change.new == 'add a decreasing number of monomers starting at:':
    monomerrefilldep_widgets.children = ([numberchoicedep_widgets])
    monomer_label.children = ([refillnumberdecreaselabel])

monomerrefill.observe(changemonomerstatus)


def batch_run():
  # Set parameters
  params_from_dict()
  # Init lookup tables
  global L
  global R
  global N_RANDS
  global rands
  global break_prob_lookup_table
  global poisson_dict

  rands = []
  L = True
  R = False
  N_RANDS = 1000
  break_prob_lookup_table = {}
  poisson_dict = {}
  # Get datetime for filename
  timestamp = str(datetime.datetime.now()).replace(" ","_")
  # Export parameter file
  dict_to_csv(params,f"{wdir}/{folder}/{batch_name}_{timestamp}_params.csv")
  # Make reactables
  react_bag = Reactables(make_pool(POOL_SIZE))
  # Run simulation
  react_bag.simulate(POOL_SIZE, ITERATIONS, METHOD)
  # Get results
  stats = react_bag.get_stats()
  # Export data file
  stats.to_csv(f"{wdir}/{folder}/{batch_name}_{timestamp}_data.csv")
  

def params_from_dict():
  global BASE_BOND_BREAK_PROBABILITY
  global HOMOCHIRAL_BREAK_FACTOR
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR
  global LENGTH_FACTOR
  global N
  global LAMBDA
  global HOMOCHIRAL_BREAK_FACTOR_LEFT
  global HOMOCHIRAL_BREAK_FACTOR_RIGHT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT
  global POOF_CHANCE
  global BOND_PROB
  global POOL_SIZE
  global ITERATIONS
  global METHOD
  global break_prob_lookup_table
  global poisson_dict
  global POISSON_FACTOR
  global FUSION
  global REFILL

  BASE_BOND_BREAK_PROBABILITY = params["BASE_BOND_BREAK_PROBABILITY"]
  HOMOCHIRAL_BREAK_FACTOR = params["HOMOCHIRAL_BREAK_FACTOR"]
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR = params["HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR"]
  LENGTH_FACTOR = params["LENGTH_FACTOR"]
  N = params["N"]
  LAMBDA = params["LAMBDA"]
  HOMOCHIRAL_BREAK_FACTOR_LEFT = params["HOMOCHIRAL_BREAK_FACTOR_LEFT"]
  HOMOCHIRAL_BREAK_FACTOR_RIGHT = params["HOMOCHIRAL_BREAK_FACTOR_RIGHT"]
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT = params["HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT"]
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT = params["HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT"]
  POOF_CHANCE = params["POOF_CHANCE"]
  BOND_PROB = params["BOND_PROB"]
  POOL_SIZE = params["POOL_SIZE"]
  ITERATIONS = params["ITERATIONS"]
  METHOD = params["METHOD"]
  POISSON_FACTOR = params["POISSON_FACTOR"]
  FUSION = params["FUSION"]
  REFILL = params["REFILL"]

# Ok, here it is!
def batch_recur(variables, ranges, reps=1, depth=0):
  """
  Recursively run a batch of simulations.

  parameters:

  variables (list):
  A list of string literals referring to the parameters that should vary in the 
    batch. The strings should be keys in the params dictionary.

  ranges (list):
  A list of lists containing the values each corresponding parameter should take
    during the batch.

  reps (int, optional):
  The number of times to run each combination of parameter values. Defaults to 1.

  depth (int, optional):
  The current layer of recursion, used within the function to keep track of its
    current status. Defaults to 0, don't change it.
  """
  # for each value that this particular parameter should take,
  if ranges == []:
    raise ValueError("Need to choose at least one parameter to be dynamic!!")
  for value in ranges[depth]:
    # Set the parameter to that value.
    params[variables[depth]] = value
    # Run the sim with these settings reps times.
    for n in range(reps):
      # But hang on, is it time to run the sim yet?
      if depth < len(variables)-1:
        # If we haven't recursed enough to set every variable, we need to go deeper.
        batch_recur(variables, ranges, reps, depth+1)
      else:
        # But if we have set everything, we can run the sim and step back up a level.
        batch_run()
        f.value += 1


#display container
display(container)

def runsim(b):
  """Method for running through one simulation"""
  continue_box.children = ([])
  #initialize all global parameters - values will be replaced so they are unimportant
  BASE_BOND_BREAK_PROBABILITY = 0.9
  HOMOCHIRAL_BREAK_FACTOR = 0.9
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR = 0.3
  LENGTH_FACTOR = 0.6
  N=40
  LAMBDA=6
  HOMOCHIRAL_BREAK_FACTOR_LEFT = 0.0
  HOMOCHIRAL_BREAK_FACTOR_RIGHT = 0.9
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT = 0.3
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT = 0.3
  POOF_CHANCE = 0.3333
  BOND_PROB = 0.3333
  POOL_SIZE = 100
  ITERATIONS = 100
  METHOD = 'standard'
  POISSON_FACTOR = 1.3
  FUSION = False
  REFILL = ["no refill"]

  #make a global dictionary of all the parameters
  global params
  #Housekeeping: Store parameters in a dictionary for exporting later

  params = {"BASE_BOND_BREAK_PROBABILITY":BASE_BOND_BREAK_PROBABILITY, 
        "HOMOCHIRAL_BREAK_FACTOR":HOMOCHIRAL_BREAK_FACTOR,
        "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR,
        "LENGTH_FACTOR":LENGTH_FACTOR,
        "N":N,
        "LAMBDA":LAMBDA,
        "HOMOCHIRAL_BREAK_FACTOR_LEFT":HOMOCHIRAL_BREAK_FACTOR_LEFT,
        "HOMOCHIRAL_BREAK_FACTOR_RIGHT":HOMOCHIRAL_BREAK_FACTOR_RIGHT,
        "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT,
        "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT":HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT,
        "POOF_CHANCE":POOF_CHANCE,
        "BOND_PROB":BOND_PROB,
        "POOL_SIZE":POOL_SIZE,
        "ITERATIONS":ITERATIONS,
        "METHOD":METHOD,
        "POISSON_FACTOR":POISSON_FACTOR,
        "FUSION":FUSION,
        "REFILL":REFILL}
  #initialize a list of parameters that will be dynamic
  global to_change
  to_change = []

  #initialize a list of the values of these parameters that will be tested
  global ranges
  ranges = []

  #now make a dataframe of all the values that dashboard widgets provides(monomerrefill parameters are wierd and won't be in here

  #column for variable
  variables = ["BASE_BOND_BREAK_PROBABILITY","HOMOCHIRAL_BREAK_FACTOR","HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR", "LENGTH_FACTOR", "N", "LAMBDA","HOMOCHIRAL_BREAK_FACTOR_LEFT", "HOMOCHIRAL_BREAK_FACTOR_RIGHT", 
   "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT", "HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT", "POOF_CHANCE", "BOND_PROB", "POOL_SIZE", "ITERATIONS", "POISSON_FACTOR", "FUSION"]
  #column for whether or not they will be dynamic
  variablestobedynamic = [checkbbbp.value,checkhbf.value, checkhnif.value,
                    checklf.value, checkn_val.value,
                    checklamb.value, checkhbfleft.value,
                    checkhbfright.value, checkhnifleft.value,
                    checkhnifright.value, checkpc.value, checkbp.value, checkps.value, checkiterns.value, checkpf.value, checkfusion.value]
  #column for the static value of these
  variablesstaticvalue = [staticvaluebbbp.value,staticvaluehbf.value, staticvaluehnif.value,
                    staticvaluelf.value, staticvaluen_val.value,
                    staticvaluelamb.value, staticvaluehbfleft.value,
                    staticvaluehbfright.value, staticvaluehnifleft.value,
                    staticvaluehnifright.value, staticvaluepc.value, staticvaluebp.value, staticvalueps.value, staticvalueiterns.value, 
                    staticvaluepf.value,staticvaluefusion.value]
  #column for range value
  variablesrangevalue = [rangevaluebbbp.value, rangevaluehbf.value,rangevaluehnif.value,
                    rangevaluelf.value, rangevaluen_val.value,
                    rangevaluelamb.value, rangevaluehbfleft.value,
                    rangevaluehbfright.value, rangevaluehnifleft.value,
                    rangevaluehnifright.value, rangevaluepc.value, rangevaluebp.value, rangevalueps.value, rangevalueiterns.value, 
                    rangevaluepf.value,polymerfusionon.value]

  #column for step value
  variablesstepvalue = [stepvaluebbbp.value, stepvaluehbf.value, stepvaluehnif.value,
                    stepvaluelf.value, stepvaluen_val.value,
                    stepvaluelamb.value, stepvaluehbfleft.value,
                    stepvaluehbfright.value, stepvaluehnifleft.value,
                    stepvaluehnifright.value, stepvaluepc.value, stepvaluebp.value, stepvalueps.value, stepvalueiterns.value, 
                    stepvaluepf.value,polymerfusionoff.value]

  #deal with method variable 
  params["METHOD"] = method_picker.value


  #create dataframe
  data = {'variable':  variables, 'dynamic': variablestobedynamic,
      'staticvalue': variablesstaticvalue, 'rangevalue': variablesrangevalue, 'stepvalue': variablesstepvalue}
  df = pd.DataFrame(data, columns = ['variable','dynamic',
      'staticvalue', 'rangevalue', 'stepvalue'],)
  
  #make rows be names by variable
  df.set_index("variable",inplace = True)

  #loop through all parameters in dict
  for parameter in params:
    #don't go through parameters that handled a little differently
    exceptions = ["METHOD","REFILL"]#"REFILLRANDOM", "REFILLPERCENT","REFILLNUMBER", "REFILLNUMBERDECREASE", "REFILLNORMAL", "REFILLPERCENTDECREASE"]
    if exceptions.count(parameter)!=0:
      continue

    #if parameter is selected to be static
    if df.loc[parameter,"dynamic"] == False:
      #add static value to dictionary
      params[parameter] = df.loc[parameter,"staticvalue"]

    else:
      #otherwise add parameter to to_change
      to_change.append(parameter)
      #if parameter is Fusion
      if parameter == "FUSION":
        #make list of options selected to be rotated through
        options = []
        if polymerfusionon.value == True:
          options.append(True)
        if polymerfusionoff.value == True:
          options.append(False)
        #add this to ranges
        ranges.append(options)
      else:
        #otherwise make array with range and step values
        array = np.arange(start = df.loc[parameter,"rangevalue"][0], stop = df.loc[parameter,"rangevalue"][1], step = df.loc[parameter, "stepvalue"])
        #add this array to ranges
        ranges.append(array.tolist())

  #now deal with the monomer methods
  values = []
  array = []
  #first deal with the options that don't have additional dynamic options
  if monomerrefill.value =='refill monomers so that # of reactants = pool size':
    #if selected = make params value true
    params["REFILL"] = ["refill normal"]

  if monomerrefill.value == 'add random number of monomers':
     params["REFILL"] = ["refill random"]
  
  #next deal with number/percent options
  if monomerrefill.value == 'add a specified number of monomers':
    if checkmonomernumber.value == True:
      #if dynamic = add to the to be changed list
      to_change.append("REFILL")
      #make array of values with range and step values
      array = np.arange(start = numberrange.value[0], stop = numberrange.value[1],step = numberstep.value)
      refillmethod = "refill number"
    else:
      #otherwise change params value to True and static value
      params["REFILL"] = ["refill number", numberslider.value]
  #repeat for other options
  if monomerrefill.value == 'add a specified percent of the difference':
     if checkmonomerpercent.value == True:
      to_change.append("REFILL")
      array = np.arange(start = percentrange.value[0], stop = percentrange.value[1],step = percentstep.value)
      refillmethod = "refill percent"
     else:
      params["REFILL"] = ["refill percent",percentslider.value]
  if monomerrefill.value == 'add a decreasing percent of the difference starting at:':
    if checkmonomerpercent.value == True:
      to_change.append("REFILL")
      array = np.arange(start = percentrange.value[0], stop = percentrange.value[1], step = percentstep.value)
      refillmethod = "refill percent decrease"
    else:
      params["REFILL"] = ["refill percent decrease", percentslider.value]

  if monomerrefill.value == 'add a decreasing number of monomers starting at:':
    if checkmonomernumber.value == True:
      to_change.append("REFILL")
      array = np.arange(start = numberrange.value[0], stop = numberrange.value[1],step = numberstep.value)
      refillmethod = "refill number decrease"
    else:
      params["REFILL"] = ["refill number decrease", numberslider.value]
  #if length of array not empty = meaning a dynamic options was chosen
  if len(array) >= 1:
    #go through array
    for value in range(len(array)):
      #add  a tuple of [True, value] to values
      values.append([refillmethod, array[value]])
    #add values to ranges
    ranges.append(values)
  
  # now that we have all our parameter taken care of

  #initialize variables for exporting
  global folder
  folder = folder_widget.value
  global batch_name
  batch_name = batch_name_widget.value
  #Check if there is already something with that name
  if len(glob.glob(f'/content/drive/Shared drives/Homochirality/{folder}/{batch_name}_*.csv')) > 0:
    raise ValueError("That batch name is not available.")
  #make a dictionary of all the check boxes
  # And how many times to run it with each combination of settings
  global reps
  reps = repswidget.value
  # Figure out how many runs it's gonna take
  run_lens = [len(range) for range in ranges]
  n_combs = np.prod(run_lens)
  global n_runs
  n_runs = n_combs * (reps**len(run_lens))
  continuelabel = ipywidgets.Label("This will create " + str(2*n_runs) + " files. Press continue to continue or reset you parameters and click go")
  continue_box.children = ([continuelabel, continuebutton])

def complete_run(b):
  #if user chooses to complete run..
  #show progress bar
  global f 
  f = ipywidgets.IntProgress(min = 0, max = n_runs)
  display (f)
  #run batch
  batch_recur(to_change, ranges, reps)
  print("Done!")

#display continue box
display(continue_box)
#method to read go button and runsim
gobutton.on_click(runsim)
#method to run when continue button is used
continuebutton.on_click(complete_run)



VBox()

In [58]:
def f(x) :
  if x < 20: 
    y = 0.00
  if x >= 20: 
    y = 0.00128 * x + 0.005
  skew = 0.5 + y
  return skew
length = []
skew = [] 
for i in range(50):
  length.append(i)
  skew.append(f(i))
print(length)
print(skew)

fig = px.scatter(y = skew,x = length)

fig.show()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5306, 0.53188, 0.53316, 0.53444, 0.53572, 0.537, 0.53828, 0.53956, 0.54084, 0.54212, 0.5434, 0.54468, 0.54596, 0.54724, 0.54852, 0.5498, 0.55108, 0.55236, 0.55364, 0.55492, 0.5562, 0.55748, 0.55876, 0.56004, 0.56132, 0.5626, 0.56388, 0.56516, 0.56644, 0.56772]
